In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:35451")
client

## Flatten and index values with their column ids

In [ ]:
# import ftfy
# import numpy as np
# import pandas as pd
# import dask.dataframe as dd

# df = dd.read_csv("../../data/raw/viznet/*.csv")
# df["idx"] = 1
# df["idx"] = df["idx"].cumsum()


# def eval_with_catch(x):
#     try:
#         return list(set(eval(x)))
#     except Exception:
#         return []


# def clean_str(x):
#     try:
#         return ftfy.fix_text(x)
#     except:
#         return ""


# df["data"] = df["data"].apply(func=eval_with_catch, meta=("data", np.array([""])))
# df = df.explode("data")
# df["data"] = df["data"].apply(lambda x: clean_str(x), meta=("data", str))
# df = df.replace("", np.nan)
# df = df.dropna(subset=["data"])
# df.to_csv("../../data/train/viznet/indexed/all_indexed.csv", single_file=True, index=False)

In [ ]:
# df = dd.read_csv("../../data/train/viznet/indexed/all_indexed.csv")

## Convert values to patterns => Flatten and index pattern with their column ids

In [ ]:
# pdf = df.assign(
#     data=df.data.str.replace("", "").replace(r"[A-Z]", "A", True)
#     .replace(r"[a-z]", "a", True)
#     .replace(r"[0-9]", "0", True)
# )

# pdf.to_csv("../../data/train/viznet/indexed/pattern_indexed.csv", single_file=True, index=False)

In [ ]:
import ftfy
import numpy as np
import pandas as pd
import dask.dataframe as dd

pdf = dd.read_csv("../../data/train/viznet/indexed/pattern_indexed.csv")

In [ ]:
pdf = pdf.assign(
    data=pdf.data.str.replace("", "").replace(r"[A-Z]+", "A", True)
    .replace(r"[a-z]+", "a", True)
    .replace(r"[0-9]+", "0", True)
)

In [ ]:
pdf["idx"] = pdf["idx"].apply(lambda x: [x], meta=("idx", np.array([0])))
pdf = pdf.persist()

agg_df = pdf.groupby("data").agg({"idx": "sum"})
agg_df = agg_df.repartition(npartitions=64)
agg_df = agg_df.assign(data=lambda x: x.index)

In [ ]:
agg_df.to_json("../../data/train/viznet/pattern_n_cooc.jl", orient="records", compute=True)